In [5]:
#Importing the necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [7]:
# Getting the dataset
data = pd.read_csv('ShillBiddingDataset.csv')
# Converting the dataset to a dataframe
df = pd.DataFrame(data)
# Displaying the dataset
df

,Record_ID,Auction_ID,Bidder_ID,Bidder_Tendency,Bidding_Ratio,Successive_Outbidding,Last_Bidding,Auction_Bids,Starting_Price_Average,Early_Bidding,Winning_Ratio,Auction_Duration,Class
0,1,732,_***i,0.200000,0.400000,0.0,0.000028,0.000000,0.993593,0.000028,0.666667,5,0
1,2,732,g***r,0.024390,0.200000,0.0,0.013123,0.000000,0.993593,0.013123,0.944444,5,0
2,3,732,t***p,0.142857,0.200000,0.0,0.003042,0.000000,0.993593,0.003042,1.000000,5,0
3,4,732,7***n,0.100000,0.200000,0.0,0.097477,0.000000,0.993593,0.097477,1.000000,5,0
4,5,900,z***z,0.051282,0.222222,0.0,0.001318,0.000000,0.000000,0.001242,0.500000,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6316,15129,760,l***t,0.333333,0.160000,1.0,0.738557,0.280000,0.993593,0.686358,0.888889,3,1
6317,15137,2481,s***s,0.030612,0.130435,0.0,0.005754,0.217391,0.993593,0.000010,0.878788,7,0
6318,15138,2481,h***t,0.055556,0.043478,0.0,0.015663,0.217391,0.993593,0.015663,0.000000,7,0
6319,15139,2481,d***d,0.076923,0.086957,0.0,0.068694,0.217391,0.993593,0.000415,0.000000,7,0


In [9]:
# Selecting all the features minus Record_ID, Auction_ID, Bidder_ID and Class
X = df.iloc[:, 3:12]
# The target variable, Class.
y = df.iloc[:,-1]

# Training and testing the dataset.
# The test size will use 20% to test and 80% to train.
# The random_state is a signature my proffessor likes to use.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=110)


In [11]:
#finding the splits
def splits(x):
    #store results
    mid_points = []
    # Sort
    x_sorted = sorted(x)
    # Iterate over using the index 
    for i in range(0, len(x)-1):
        mid_point = (x_sorted[i] + x_sorted[i+1])/2
        mid_points.append(mid_point)
    return mid_points

In [13]:
# creating a mask 
def split_data(x,y,split_point):
    mask = x >= split_point
    anti_mask = x < split_point
    x_true = x[mask]
    y_true = y[mask]
    x_false = x[anti_mask]
    y_false = y[anti_mask]
    return x_true,y_true,x_false,y_false

In [15]:
# I'm using gini 
def gini(y):
    
    # I kept having empty errors or 0 errors, so automatically return 1 if empty.
    if len(y) == 0:
        return 1 
    
    # Calculate the fraction of sample 1
    p1 = np.sum(y) / len(y)
    #  Calculate the fraction of sample 0
    p0 = 1 - p1
    # summation of the two 
    impurity = 1 - (p0**2 + p1**2)
    return impurity

In [17]:
def get_best_split(data):
    data = data.to_numpy()
    n_features = data.shape[1]-1
    list_errors = {}

    min_err = 100000000
    target_idx = - 1
    feature_ind = [3,4,5,6,7,8,9,10,11,12]

    all_errs = {}
    for feature_idx in (feature_ind):
        x = data[:,feature_idx]
        y = data[:,target_idx]
        split_pts = splits(x)
        
        for point in split_pts:
            x_true, y_true, x_false, y_false = split_data(x,y,point)
            gini_true = gini(y_true)
            gini_false = gini(y_false)
            summed_gini = gini_true + gini_false
            
            if summed_gini < min_err:
                min_err = summed_gini
                best_feature = feature_idx
                best_point = point

    return best_point, best_feature
    

In [19]:
# Testing to see if it worked
result = get_best_split(data)
result

(0.5, 12)

In [20]:
# Full function for a basic decision tree.
def basic_dt(data, sample):
    best_point, best_feature = get_best_split(data)
    
    if sample[best_feature] >= best_point:
        return data[data.iloc[:,best_feature] >= best_point]
    else:
        return data[data.iloc[:,best_feature] < best_point]

In [21]:
# Booststrapping 
def bootstrap(data):
    n_samples = data.shape[0]
    sampled_data = []
    for i in range(n_samples):
        random_index = np.random.randint(0,n_samples)
        sampled_data.append(data.iloc[random_index])
    return np.array(sampled_data)

In [22]:
# Had to get oob samples as well
def bs_oob_samples(data):
    bs = bootstrap(data)
    nuique_bs_samples = np.unique(bs, axis=0)
    oob_samples = []
    for sample in data:
        if sum(np.all(sample == nuique_bs_samples, axis= 1)) == 0:
            oob_samples.append(sample)
    return bs,np.array(oob_samples)

In [23]:
# Implementing random forest classifier
def random_forest_classifier(no_of_trees, depth, no_samples_per_split ,no_samples_per_node):
    all_trees = []
# I couldn't figure out the logic  to implement the random forest classifier...

In [49]:
# calling random forest classifier function from sklearn 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, f1_score
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=110)
# training the model
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=110)

In [47]:
# predictions
y_pred = rf_classifier.predict(X_test)

In [51]:
# confusion matirx
confusion_m = confusion_matrix(y_test,y_pred)
confusion_m

array([[1130,    7],
       [   1,  127]])

In [55]:
# flatten the array.
confusion_m.ravel()

array([1130,    7,    1,  127])

In [59]:
TN , FP , FN , TP = confusion_m.ravel()

In [65]:
 # TN (True Negative): Correctly predicted negative cases
TN

1130

In [67]:
# FP (False Positive): Incorrectly predicted positive cases
FP

7

In [69]:
# FN (False Negative): Incorrectly predicted negative cases
FN

1

In [71]:
# TP (True Positive): Correctly predicted positive cases
TP

127

In [75]:
# calculating accuracy
accuracy = (TP + TN) / confusion_m.ravel().sum()
accuracy*100

99.36758893280633

In [85]:
# calculating precision
precision = TP / (TP+FP)
precision * 100

94.77611940298507

In [89]:
# calculating recall
recall = TP / (TP + FN)
recall

0.9921875

In [87]:
# calculating F1 score
F1_score = 2 * (precision* recall) / (precision + recall)
F1_score * 100

96.94656488549617

In [99]:
# Number of frauds
data[data['Class']>0].shape[0]

675

In [97]:
# Number of normal
data[data['Class']==0].shape[0]

5646